In [44]:
import pandas as pd
from dotenv import load_dotenv
import os
import time
from google.oauth2 import service_account
from googleapiclient.discovery import build

load_dotenv()

gtm_id = os.environ["GTM_ACCOUNT_ID"]

# Excel

In [61]:
xlsx_names = ['kyobo', 'library', 'scholar', 'story', 'tree']
service_names = {
    '교보문고_renewal': 'kyobo',
    '전자도서관': 'library',
    '스콜라': 'scholar',
    '스토리': 'story',
    '리딩트리': 'tree'
}

def get_xlsx():
    res = {}
    for name in service_names.values():
        res[name]=pd.ExcelFile(f'./xlsx/{name}.xlsx')

    return res

def get_sheet_name(excel):
    sheet_names = excel.sheet_names
    return sheet_names

def read_sheet(excel_file, sheet_name):
    return pd.read_excel(excel_file, sheet_name=sheet_name)

# GTM

In [62]:
# Get credentials
def get_credentials():
    SERVICE_ACCOUNT_FILE = './xlsx/gtm-api-409502-6f8362f190d7.json'

    credentials_container = service_account.Credentials.from_service_account_file(
        SERVICE_ACCOUNT_FILE,
        scopes=["https://www.googleapis.com/auth/tagmanager.edit.containers"]
    )

    return credentials_container

credentials = get_credentials()

# Get service
def get_service():
    service = build('tagmanager', 'v2', credentials = credentials)

    return service

service = get_service()

# Get container
def get_container():
    parent = f"accounts/{gtm_id}"

    containers = service.accounts().containers().list(
        parent=parent,
    ).execute()

    return containers['container']

container_origin = get_container()
containers = {}
for i in container_origin:
    containers[i['name']] = i


# Set Container

In [63]:
# 교보문고_renewal, 전자도서관, 스콜라, 스토리, 리딩트리
service_name = '교보문고_renewal'
container = containers[service_name]

xlsxs = get_xlsx()
xlsx = xlsxs[service_names[service_name]]

# Get workspace
def get_workspaces():
    parent=container['path']

    workspace = service.accounts().containers().workspaces().list(
        parent=parent
    ).execute()

    return workspace['workspace']

workspace_origin = get_workspaces()

workspaces = {}
for i in workspace_origin:
    workspaces[i['name']] = i

workspace = workspaces['new']

# Get Elements

In [64]:
def get_variables():
    parent=workspace['path']

    containers = service.accounts().containers().workspaces().variables().list(
        parent=parent
    ).execute()

    return containers

def get_trigger_list():
    parent=workspace['path']

    try:
        response = service.accounts().containers().workspaces().triggers().list(
            parent=parent
        ).execute()

        print(f"Listed triggers: {response['trigger']}")
        return response
    except Exception as e:
        print(f"An error occurred: {e}")
        return e

def get_tag_list():
    parent=workspace['path']

    try:
        response = service.accounts().containers().workspaces().tags().list(
            parent=parent
        ).execute()

        print(f"Listed triggers: {response['tag']}")
        return response
    except Exception as e:
        print(f"An error occurred: {e}")
        return e


# Variables

In [7]:
def create_gtm_variable(request_body):
    parent=workspace['path']

    try:
        response = service.accounts().containers().workspaces().variables().create(
            parent=parent,
            body=request_body
        ).execute()

        print(f"Created variable: {response['name']}")
        return response
    except Exception as e:
        print(f"An error occurred: {e}")
        print(f"Above error occurred with: {request_body['name']}")
        return e

def list_gtm_variable():
    parent=workspace['path']

    try:
        response = service.accounts().containers().workspaces().variables().list(
            parent=parent
        ).execute()

        print(f"Listed variables: {response['variable']}")
        return response
    except Exception as e:
        print(f"An error occurred: {e}")

In [96]:
def create_aev_variable(sheet):
    error = 0
    for row in sheet.iloc():
        if (row[1] == '요소 속성'):
            parameter = [
                {
                    "type": "template",
                    "key": "elementSelector",
                    "value": "ATTRIBUTE"
                },
                {
                    'type': 'template',
                    'key': 'attribute',
                    'value': row[2]
                },
                {
                    'type': 'boolean',
                    'key': 'setDefaultValue',
                    'value': 'false'
                },
            ]

        request_body = {
            "name": row[0],
            "type": 'aev',
            "parameter": parameter
        }

        try:
            create_gtm_variable(request_body)
            time.sleep(5)
        except Exception as e:
            error += 1

    print(f"Total Error: {error}")

In [97]:
def create_dom_variable(sheet):
    error = 0
    for row in sheet.iloc():
        if (row[1] == 'CSS 선택도구'):
            parameter = [
                {
                    "type": "template",
                    "key": "elementSelector",
                    "value":row[2] 
                },
                {
                    'type': 'template',
                    'key': 'selectorType',
                    'value': 'CSS'
                }
            ]

        request_body = {
            "name": row[0],
            "type": 'd',
            "parameter": parameter
        }

        try:
            create_gtm_variable(request_body)
            time.sleep(5)
        except Exception as e:
            error += 1

    print(f"Total Error: {error}")

In [107]:
def create_table_variable(sheet):
    error = 0
    request_body = {}

    for row in sheet.iloc():
        if (type(row[0]) is str):
            if (len(request_body) != 0):
                try:
                    create_gtm_variable(request_body)
                    time.sleep(5)
                except Exception as e:
                    error += 1

            request_body['name'] = row[0]
            formatValue = {}
            parameter = []
            map_data = []
            parameter.append({
                'type': 'list',
                'key': 'map',
                'list': map_data
            })
            request_body['parameter'] = parameter

            if (row[1] == '참고표'):
                request_body['type'] = 'smm'
            if (row[1] == '정규식 표'):
                request_body['type'] = 'remm'
                request_body['parameter'].append({'type': 'boolean', 'key': 'fullMatch', 'value': 'true'})
                request_body['parameter'].append({'type': 'boolean', 'key': 'replaceAfterMatch', 'value': 'true'})
                request_body['parameter'].append({'type': 'boolean', 'key': 'ignoreCase', 'value': 'true'})

        if (type(row[5]) is str):
            parameter.append({
                'type': 'boolean',
                'key': 'setDefaultValue',
                'value': 'true'
            })
            parameter.append({
                'type': 'template',
                'key': 'defaultValue',
                'value': row[6]
            })

        if (type(row[7]) is str):
            if (row[7][:5] == 'undef'):
                formatValue['convertUndefinedToValue'] = {
                    'type': 'template',
                    'value': row[8]
                }
            if (row[7][:5] == 'false'):
                formatValue['convertFalseToValue'] = {
                    'type': 'template',
                    'value': row[8]
                }
            if (row[7][:4] == 'null'):
                formatValue['convertNullToValue'] = {
                    'type': 'template',
                    'value': row[8]
                }
            

        if (type(row[2]) is str):
            parameter.append({
                'type': 'template',
                'key': 'input',
                'value': row[2]
            })

        if (type(row[3]) is str):
            input_data = {
                'type': 'map',
                'map': [
                    {
                        'type': 'template',
                        'key': 'key',
                        'value': row[3]
                    },
                    {
                        'type': 'template',
                        'key': 'value',
                        'value': row[4]
                    }
                ]
            }
            map_data.append(input_data)

        
        request_body['formatValue'] = formatValue

    try:
        create_gtm_variable(request_body)
        time.sleep(5)
    except Exception as e:
        error += 1

    print(f"Total Error: {error}")

In [83]:
def create_gtes_variable(sheet):
    error = 0
    request_body = {}
    for row in sheet.iloc():
        if (type(row[0]) is str):
            if (len(request_body) != 0):
                try:
                    create_gtm_variable(request_body)
                    time.sleep(5)
                except Exception as e:
                    error += 1

            request_body['name'] = row[0]
            request_body['type'] = 'gtes'
            eventSettingsTable = []
            userProperties = []

        if (type(row[2]) is str):
            event_data = {
                'type': 'map',
                'map': [
                    {
                        'type': 'template',
                        'key': 'parameter',
                        'value': row[2]
                    },
                    {
                        'type': 'template',
                        'key': 'parameterValue',
                        'value': row[3]
                    }
                ]
            }
            eventSettingsTable.append(event_data)
        if (type(row[4]) is str):
            user_data = {
                'type': 'map',
                'map': [
                    {
                        'type': 'template',
                        'key': 'name',
                        'value': row[4]
                    },
                    {
                        'type': 'template',
                        'key': 'value',
                        'value': row[5]
                    }
                ]
            }
            userProperties.append(user_data)

        request_body['parameter'] = [{
            'type': 'list',
            'key': 'eventSettingsTable',
            'list': eventSettingsTable
        },
        {
            'type': 'list',
            'key': 'userProperties',
            'list': userProperties
        }]

    try:
        create_gtm_variable(request_body)
        time.sleep(5)
    except Exception as e:
        error += 1

    print(f"Total Error: {error}")

In [84]:
def create_url_variable(sheet):
    error = 0
    for row in sheet.iloc():
        if (row[2] == '검색어'):
            parameter = [
                {
                "type": "template",
                "key": "component",
                "value": "QUERY"
                },
                {
                    'type': 'template',
                    'key': 'queryKey',
                    'value': row[3]
                }
            ]
            type = 'u'

        request_body = {
            "name": row[0],
            "type": type,
            "parameter": parameter
        }

        try:
            create_gtm_variable(request_body)
            time.sleep(5)
        except Exception as e:
            error += 1

    print(f"Total Error: {error}")

In [85]:
def create_dataLayer_variable(sheet):
    error = 0
    for row in sheet.iloc():
        if (row[1] == '상수'):
            parameter = [{
                'type': 'template',
                'key': 'value',
                'value': row[2],
            }]
            type = 'c'
        elif (row[1] == '맞춤 자바스크립트'):
            parameter = [{
                'type': 'template',
                'key': 'javascript',
                'value': row[2],
            }]
            type = 'jsm'
        elif (row[1] == '당사 쿠키'):
            parameter = [{
                'type': 'template',
                'key': 'name',
                'value': row[2],
            }]
            type = 'k'
        else:
            parameter = [
                {
                    'type': 'template',
                    'key': 'name',
                    'value': row[2],
                },
                {
                    'type': 'integer',
                    'key': 'dataLayerVersion',
                    'value': 2,
                }
                ]
            type = 'v'

        request_body = {
            'name': row[0],
            'type': type,
            'parameter': parameter,
        }

        try:
            create_gtm_variable(request_body)
            time.sleep(5)
        except Exception as e:
            error += 1

    print(f"Error: {error}")


In [182]:
sheet_names = get_sheet_name(xlsx)

for index, i in enumerate(sheet_names):
  print(f"{index}: {i}")

0: 변수(당사쿠키)
1: 변수(데이터 영역)
2: 변수(URL)
3: 변수(자동이벤트 변수)
4: 변수(DOM 요소)
5: 변수(맞춤JS)
6: 변수(참고표)
7: 변수(이벤트 매개변수)
8: 트리거(맞춤 이벤트)
9: 트리거(페이지뷰)
10: 트리거(스크롤 깊이)
11: 트리거(링크 클릭)(Web)
12: 트리거(링크 클릭)(App)
13: 트리거(모든 요소 클릭)(Web)
14: 트리거(모든 요소 클릭)(App)
15: 태그(맞춤HTML)
16:  태그(Web)
17:  태그(App 이벤트)


In [108]:
# sheet_constants = read_sheet(xlsx, sheet_names[0])
# sheet_cookie = read_sheet(xlsx, sheet_names[1])
# sheet_data_layer = read_sheet(xlsx, sheet_names[1])
# sheet_url = read_sheet(xlsx, sheet_names[1])
# sheet_aev = read_sheet(xlsx, sheet_names[4])
# sheet_dom = read_sheet(xlsx, sheet_names[2])
# sheet_custom_js = read_sheet(xlsx, sheet_names[3])
sheet_table = read_sheet(xlsx, sheet_names[6])
# sheet_gtes = read_sheet(xlsx, sheet_names[5])

# create_dataLayer_variable(sheet_constants)
# create_dataLayer_variable(sheet_cookie)
# create_dataLayer_variable(sheet_data_layer)
# create_dataLayer_variable(sheet_custom_js)
# create_url_variable(sheet_url)
# create_aev_variable(sheet_aev)
# create_dom_variable(sheet_dom)
# create_gtes_variable(sheet_gtes)
create_table_variable(sheet_table)

/var/folders/xq/qf2k1k454qb_jfv62h4qfhn00000gn/T/ipykernel_45950/3354141961.py:6: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  if (type(row[0]) is str):
/var/folders/xq/qf2k1k454qb_jfv62h4qfhn00000gn/T/ipykernel_45950/3354141961.py:14: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  request_body['name'] = row[0]
/var/folders/xq/qf2k1k454qb_jfv62h4qfhn00000gn/T/ipykernel_45950/3354141961.py:25: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[po

An error occurred: <HttpError 400 when requesting https://tagmanager.googleapis.com/tagmanager/v2/accounts/6003640738/containers/90556056/workspaces/43/variables?alt=json returned "Returned an error response for your request.". Details: "[{'message': 'Returned an error response for your request.', 'domain': 'global', 'reason': 'badRequest'}]">
Above error occurred with: TD - item_list_id
An error occurred: <HttpError 400 when requesting https://tagmanager.googleapis.com/tagmanager/v2/accounts/6003640738/containers/90556056/workspaces/43/variables?alt=json returned "Returned an error response for your request.". Details: "[{'message': 'Returned an error response for your request.', 'domain': 'global', 'reason': 'badRequest'}]">
Above error occurred with: TD - item_list_name
An error occurred: <HttpError 400 when requesting https://tagmanager.googleapis.com/tagmanager/v2/accounts/6003640738/containers/90556056/workspaces/43/variables?alt=json returned "Returned an error response for your

/var/folders/xq/qf2k1k454qb_jfv62h4qfhn00000gn/T/ipykernel_45950/3354141961.py:77: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  'value': row[3]
/var/folders/xq/qf2k1k454qb_jfv62h4qfhn00000gn/T/ipykernel_45950/3354141961.py:82: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  'value': row[4]


Created variable: TD - 메뉴 - 서비스탭 명칭
Created variable: TD - 검색 팝업 - 링크 이름
Created variable: TD - 검색 팝업 - 버튼 이름
Created variable: TD - gbCode 명칭
Created variable: TD - target 명칭
Created variable: TD - 상품 상세 공유 수단


/var/folders/xq/qf2k1k454qb_jfv62h4qfhn00000gn/T/ipykernel_45950/3354141961.py:54: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  'value': row[8]


An error occurred: <HttpError 400 when requesting https://tagmanager.googleapis.com/tagmanager/v2/accounts/6003640738/containers/90556056/workspaces/43/variables?alt=json returned "Returned an error response for your request.". Details: "[{'message': 'Returned an error response for your request.', 'domain': 'global', 'reason': 'badRequest'}]">
Above error occurred with: TD - 운영/개발 서버 구분
Created variable: TD - item_list_tab(PICKS)
An error occurred: <HttpError 400 when requesting https://tagmanager.googleapis.com/tagmanager/v2/accounts/6003640738/containers/90556056/workspaces/43/variables?alt=json returned "Returned an error response for your request.". Details: "[{'message': 'Returned an error response for your request.', 'domain': 'global', 'reason': 'badRequest'}]">
Above error occurred with: TD - 마이룸 - item_list_name
An error occurred: <HttpError 400 when requesting https://tagmanager.googleapis.com/tagmanager/v2/accounts/6003640738/containers/90556056/workspaces/43/variables?alt=j

# Triggers

In [65]:
event_type_map = {
    '맞춤 이벤트': 'customEvent',
    '페이지뷰': 'pageview',
    '스크롤 깊이': 'scrollDepth',
    '클릭 - 링크만': 'linkClick',
    '클릭만 - 링크': 'linkClick',
    '클릭 - 모든 요소': 'click'
}

filter_type_map = {
    '다음과 같지 않음': 'equals',
    '같음': 'equals',   
    '포함': 'contains',
    '정규 표현식과 일치': 'matchRegex',
    "정규 표현식과 일치 ": 'matchRegex',
    '다음으로 시작 하지 않음': 'startsWith',
    '다음으로 시작': 'startsWith',
    'CSS 선택 도구와 일치': 'cssSelector',
    '다음으로 끝남': 'endsWith',
    '다음으로 끝나지 않음': 'endsWith',
}

negate_map = {
    '다음과 같지 않음': 'true',
    '같음': 'false',   
    '포함': 'false',
    '정규 표현식과 일치': 'false',
    '정규 표현식과 일치 ': 'false',
    '다음으로 시작 하지 않음': 'true',
    '다음으로 시작': 'false',
    'CSS 선택 도구와 일치': 'false',
    '다음으로 끝남': 'false',
    '다음으로 끝나지 않음': 'true',
}

def create_trigger(request_body):
    parent=workspace['path']

    try:
        response = service.accounts().containers().workspaces().triggers().create(
            parent=parent,
            body=request_body
        ).execute()

        print(f"Created : {response['name']}", end="")
        return response
    except Exception as e:
        print('\n' + f"An error occurred: {e}")
        print(f"Above error occurred with: {request_body['name']}")
        return e


In [130]:
def create_trigger_scroll(sheet):
    error = 0
    request_body = {}

    for index, row in sheet.iterrows():
        if type(row['트리거명']) == str:
            if (index != 0):
                try:
                    create_trigger(request_body)
                    time.sleep(5)
                except Exception as e:
                    error += 1

            request_body = {
                "name": row['트리거명'],
                "type": event_type_map[row['트리거 유형']],
                "parameter": [
                    {
                        'type': 'template', 
                        'key': 'verticalThresholdUnits', 
                        'value': 'PERCENT'
                    }, 
                    {
                        'type': 'template', 
                        'key': 'verticalThresholdsPercent', 
                        'value': row['비율']
                    }, 
                    {
                        'type': 'boolean', 
                        'key': 'verticalThresholdOn', 
                        'value': 'true'
                    }, 
                    {
                        'type': 'template', 
                        'key': 'triggerStartOption', 
                        'value': 'WINDOW_LOAD'
                    }, 
                    {
                        'type': 'boolean', 
                        'key': 'horizontalThresholdOn', 
                        'value': 'false'
                    }
                ],
                "filter": []
            }


        request_body['filter'].append({
            "type": filter_type_map[row['연산자(메뉴)']],
            "parameter": [
                {
                    "type": "template",
                    "key": "arg0",
                    "value": "{{" + row['변수(메뉴)'] + "}}"
                },
                {
                    "type": "template",
                    "key": "arg1",
                    "value": row['값(텍스트 입력란)']
                },
                {
                    "type": "boolean",
                    "key": "negate",
                    "value": negate_map[row['연산자(메뉴)']]
                }
            ]
        })

    try:
        create_trigger(request_body)
        time.sleep(5)
    except Exception as e:
        error += 1

    print(f"Total Error: {error}")


In [66]:
def create_trigger_click(sheet):
    error = 0
    request_body = {}

    for index, row in sheet.iterrows():
        if type(row['트리거명']) == str:
            if (index != 0):
                try:
                    create_trigger(request_body)
                    time.sleep(4)
                except Exception as e:
                    error += 1

            request_body = {
                "name": row['트리거명'],
                "type": event_type_map[row['트리거 유형']],
                'waitForTags': {'type': 'boolean', 'value': 'false'}, 
                'checkValidation': {'type': 'boolean', 'value': 'false'}, 
                'waitForTagsTimeout': {'type': 'template', 'value': '2000'}, 
                'uniqueTriggerId': {'type': 'template'},
                "filter": []
            }


        request_body['filter'].append({
            "type": filter_type_map[row['연산자(메뉴)']],
            "parameter": [
                {
                    "type": "template",
                    "key": "arg0",
                    "value": "{{" + row['변수(메뉴)'] + "}}"
                },
                {
                    "type": "template",
                    "key": "arg1",
                    "value": row['값(텍스트 입력란)']
                },
                {
                    "type": "boolean",
                    "key": "negate",
                    "value": negate_map[row['연산자(메뉴)']]
                }
            ]
        })

    try:
        create_trigger(request_body)
    except Exception as e:
        error += 1

    print(f"Total Error: {error}")


In [50]:
def create_trigger_page_view(sheet):
    error = 0
    request_body = {
        'filter': [],
    }

    for index, row in sheet.iterrows():
        if type(row['트리거명']) == str:
            if (index != 0):
                try:
                    create_trigger(request_body)
                    time.sleep(4)
                except Exception as e:
                    error += 1

            request_body = {
                "name": row['트리거명'],
                "type": event_type_map[row['트리거 유형']],
                "filter": []
            }

        request_body['filter'].append({
            "type": filter_type_map[row['연산자(메뉴)']],
            "parameter": [
                {
                    "type": "template",
                    "key": "arg0",
                    "value": "{{" + row['변수(메뉴)'] + "}}"
                },
                {
                    "type": "template",
                    "key": "arg1",
                    "value": row['값(텍스트 입력란)']
                },
                {
                    "type": "boolean",
                    "key": "negate",
                    "value": negate_map[row['연산자(메뉴)']]
                }
            ]
        })

    try:
        create_trigger(request_body)
    except Exception as e:
        error += 1

    print(f"Total Error: {error}")


In [133]:
def create_event_custom(sheet):
    error = 0
    for index, row in sheet.iterrows():
        if type(row['트리거명']) == str:
            if (index != 0):
                try:
                    create_trigger(request_body)
                    time.sleep(4)
                except Exception as e:
                    error += 1

            request_body = {
                "name": row['트리거명'],
                "type": event_type_map[row['트리거 유형']],
                "customEventFilter": [
                    {
                        "type": "equals",
                        "parameter": [
                            {
                                "type": "template",
                                "key": "arg0",
                                "value": "{{_event}}"
                            },
                            {
                                "type": "template",
                                "key": "arg1",
                                "value": row['이벤트 이름']
                            }
                        ]
                    }
                ],
                "filter": []
            }

        if type(row['변수(메뉴)']) == str:
            request_body['filter'].append({
                "type": filter_type_map[row['연산자(메뉴)']],
                "parameter": [
                    {
                        "type": "template",
                        "key": "arg0",
                        "value": "{{" + row['변수(메뉴)'] + "}}"
                    },
                    {
                        "type": "template",
                        "key": "arg1",
                        "value": row['값(텍스트 입력란)']
                    },
                    {
                        "type": "boolean",
                        "key": "negate",
                        "value": negate_map[row['연산자(메뉴)']]
                    }
                ]
            })

    try:
        create_trigger(request_body)
    except Exception as e:
        error += 1

    print(f"Total Error: {error}")


In [51]:
sheet_names = get_sheet_name(xlsx)

for index, i in enumerate(sheet_names):
    print(index, i)

0 변수(당사쿠키)
1 변수(데이터 영역)
2 변수(URL)
3 변수(자동이벤트 변수)
4 변수(DOM 요소)
5 변수(맞춤JS)
6 변수(참고표)
7 변수(이벤트 매개변수)
8 트리거(맞춤 이벤트)
9 트리거(페이지뷰)
10 트리거(스크롤 깊이)
11 트리거(링크 클릭)(Web)
12 트리거(링크 클릭)(App)
13 트리거(모든 요소 클릭)(Web)
14 트리거(모든 요소 클릭)(App)
15 태그(맞춤HTML)
16  태그(Web)
17  태그(App 이벤트)


In [67]:
# sheet_custom_event = read_sheet(xlsx, sheet_names[6])
# sheet_page_view = read_sheet(xlsx, sheet_names[9])
# sheet_scroll = read_sheet(xlsx, sheet_names[10])
sheet_link = read_sheet(xlsx, sheet_names[13])
# sheet_link_2 = read_sheet(xlsx, sheet_names[7])

# create_event_custom(sheet_custom_event)
# create_trigger_page_view(sheet_page_view)
# create_trigger_scroll(sheet_scroll)
create_trigger_click(sheet_link)
# create_trigger_click(sheet_link_2)



An error occurred: <HttpError 400 when requesting https://tagmanager.googleapis.com/tagmanager/v2/accounts/6003640738/containers/90556056/workspaces/43/triggers?alt=json returned "Returned an error response for your request.". Details: "[{'message': 'Returned an error response for your request.', 'domain': 'global', 'reason': 'badRequest'}]">
Above error occurred with: Click - 검색 팝업 - 검색어저장 켜기

An error occurred: <HttpError 400 when requesting https://tagmanager.googleapis.com/tagmanager/v2/accounts/6003640738/containers/90556056/workspaces/43/triggers?alt=json returned "Returned an error response for your request.". Details: "[{'message': 'Returned an error response for your request.', 'domain': 'global', 'reason': 'badRequest'}]">
Above error occurred with: Click - 검색 팝업 - 검색어저장 끄기

An error occurred: <HttpError 400 when requesting https://tagmanager.googleapis.com/tagmanager/v2/accounts/6003640738/containers/90556056/workspaces/43/triggers?alt=json returned "Returned an error respo

# Tags

In [68]:
tag_type_map = {
    'Google 태그': 'googtag',
    'Google 애널리틱스: GA4 이벤트': 'gaawe',
    '맞춤 HTML': 'html',
    'Firebase 이벤트 빌더': 'cvt_90556056_475',
    'Firebase 스크린 빌더': 'cvt_90556056_474',
}

triggers = get_trigger_list()

trigger_obj = {}

for i in triggers['trigger']:
    trigger_obj[i['name'].lower()] = i['triggerId']

def create_tag(request_body):
    parent=workspace['path']

    try:
        response = service.accounts().containers().workspaces().triggers().create(
            parent=parent,
            body=request_body
        ).execute()

        print(f"Created : {response['name']}", end="")
        return response
    except Exception as e:
        print('\n' + f"An error occurred: {e}")
        print(f"Above error occurred with: {request_body['name']}")
        return e


Listed triggers: [{'path': 'accounts/6003640738/containers/90556056/workspaces/43/triggers/808', 'accountId': '6003640738', 'containerId': '90556056', 'workspaceId': '43', 'triggerId': '808', 'name': 'Event - sign_up', 'type': 'customEvent', 'customEventFilter': [{'type': 'equals', 'parameter': [{'type': 'template', 'key': 'arg0', 'value': '{{_event}}'}, {'type': 'template', 'key': 'arg1', 'value': 'sign_up'}]}], 'filter': [{'type': 'equals', 'parameter': [{'type': 'template', 'key': 'arg0', 'value': '{{DLV - platform}}'}, {'type': 'template', 'key': 'arg1', 'value': 'app'}, {'type': 'boolean', 'key': 'negate', 'value': 'true'}]}], 'fingerprint': '1704351198363', 'tagManagerUrl': 'https://tagmanager.google.com/#/container/accounts/6003640738/containers/90556056/workspaces/43/triggers/808?apiLink=trigger'}, {'path': 'accounts/6003640738/containers/90556056/workspaces/43/triggers/811', 'accountId': '6003640738', 'containerId': '90556056', 'workspaceId': '43', 'triggerId': '811', 'name': 

In [54]:
{
    'name': 'GA4 - Config',
    'type': 'googtag',
    'parameter': [
        {
            'type': 'template',
            'key': 'tagId',
            'value': '{{GA4 - 측정 ID}}'
        },
        {
            'type': 'template',
            'key': 'eventSettingsVariable',
            'value': '{{GA4 - Params - 공유}}'
        }
    ],
    'tagFiringOption': 'oncePerEvent',
}

{
    'name': 'GA4 - Event - 전자상거래 - 상품 관련',
    'type': 'gaawe',
    'parameter': [
        {
            'type': 'boolean',
            'key': 'sendEcommerceData',
            'value': 'true'
        },
        {
            'type': 'template', 
            'key': 'getEcommerceDataFrom', 
            'value': 'dataLayer'
        },
        {
            'type': 'boolean', 
            'key': 'enhancedUserId', 
            'value': 'false'
        },
        {
            'type': 'template', 
            'key': 'eventName', 
            'value': '{{DLV - event_name}}'
        },
        {
            'type': 'template',
            'key': 'measurementIdOverride',
            'value': '{{GA4 - 측정 ID}}'
        },
        {
            'type': 'template',
            'key': 'eventSettingsVariable',
            'value': '{{GA4 - Params - 상품 목록 공통}}'
        }
    ],
    'tagFiringOption': 'oncePerEvent',
}

{
    'name': 'GA4 - Event - 공통 - 회원가입',
    'type': 'gaawe',
    'parameter': [
        {
            'type': 'boolean',
            'key': 'sendEcommerceData',
            'value': 'false'
        },
        {
            'type': 'boolean', 
            'key': 'enhancedUserId', 
            'value': 'false'
        },
        {
            'type': 'list',
            'key': 'eventSettingsTable',
            'list': [
                {
                    'type': 'map',
                    'map': [
                        {
                            'type': 'template', 
                            'key': 'parameter', 
                            'value': 'method'
                        },
                        {
                            'type': 'template',
                            'key': 'parameterValue',
                            'value': '{{DLV - method}}'
                        }
                    ]
                }
            ]
        },
        {
            'type': 'list',
            'key': 'userProperties',
            'list': [
                {
                    'type': 'map',
                    'map': [
                        {
                            'type': 'template',
                            'key': 'name',
                            'value': 'up_signup_method'
                        },
                        {
                            'type': 'template', 
                            'key': 'value', 
                            'value': '{{DLV - method}}'
                        }
                    ]
                }
            ]
        },
        {
            'type': 'template', 
            'key': 'eventName', 
            'value': 'sign_up'
        },
        {
            'type': 'template',
            'key': 'measurementIdOverride',
            'value': '{{GA4 - 측정 ID}}'
        }
    ],
    'tagFiringOption': 'oncePerLoad',
}

{'name': 'GA4 - Event - 공통 - 회원가입',
 'type': 'gaawe',
 'parameter': [{'type': 'boolean',
   'key': 'sendEcommerceData',
   'value': 'false'},
  {'type': 'boolean', 'key': 'enhancedUserId', 'value': 'false'},
  {'type': 'list',
   'key': 'eventSettingsTable',
   'list': [{'type': 'map',
     'map': [{'type': 'template', 'key': 'parameter', 'value': 'method'},
      {'type': 'template',
       'key': 'parameterValue',
       'value': '{{DLV - method}}'}]}]},
  {'type': 'list',
   'key': 'userProperties',
   'list': [{'type': 'map',
     'map': [{'type': 'template', 'key': 'name', 'value': 'up_signup_method'},
      {'type': 'template', 'key': 'value', 'value': '{{DLV - method}}'}]}]},
  {'type': 'template', 'key': 'eventName', 'value': 'sign_up'},
  {'type': 'template',
   'key': 'measurementIdOverride',
   'value': '{{GA4 - 측정 ID}}'}],
 'tagFiringOption': 'oncePerLoad'}

In [69]:
def create_tag_web(sheet):
    error = 0
    request_body = {}
    eventSettingsTable = {
        'type': 'list',
        'key': 'eventSettingsTable',
        'list': []
    }
    userProperties = {
        'type': 'list',
        'key': 'userProperties',
        'list': []
    }


    for index, row in sheet.iterrows():
        if type(row['태그명']) == str:
            if (index != 0):
                try:
                    request_body['parameter'].append(eventSettingsTable);
                    request_body['parameter'].append(userProperties);
                    # create_trigger(request_body)
                    print(request_body)
                    # time.sleep(4)
                except Exception as e:
                    error += 1

            request_body = {
                "name": row['태그명'],
                "type": tag_type_map[row['태그 유형']],
                "parameter": [
                    {
                        'type': 'template', 
                        'key': 'measurementIdOverride', 
                        'value': row['측정 ID(태그 ID)']
                    }
                ],
                "firingTriggerId": [],
            }
            eventSettingsTable = {
                'type': 'list',
                'key': 'eventSettingsTable',
                'list': []
            }
            userProperties = {
                'type': 'list',
                'key': 'userProperties',
                'list': []
            }

            if type(row['이벤트 이름']) == str:
                request_body['parameter'].append({
                    'type': 'template', 
                    'key': 'eventName', 
                    'value': row['이벤트 이름']
                })

        if type(row['트리거']) == str:
            request_body['firingTriggerId'].append(trigger_obj[row['트리거'].lower()])

        if type(row['이벤트 매개변수(Event Settings Variable)']) == str:
            request_body['parameter'].append({
                'type': 'template', 
                'key': 'eventSettingsVariable', 
                'value': row['이벤트 매개변수(Event Settings Variable)']
            })

        if type(row['이벤트 매개변수']) == str:
            eventSettingsTable['list'].append({
                'type': 'map',
                'map': [
                    {
                        'type': 'template', 
                        'key': 'parameter', 
                        'value': row['이벤트 매개변수']
                    },
                    {
                        'type': 'template',
                        'key': 'parameterValue',
                        'value': sheet.iloc[index][7]
                    }
                ]
            })

        if type(row['사용자 속성']) == str:
            userProperties['list'].append({
                'type': 'map',
                'map': [
                    {
                        'type': 'template',
                        'key': 'name',
                        'value': row['사용자 속성']
                    },
                    {
                        'type': 'template', 
                        'key': 'value', 
                        'value': sheet.iloc[index][9]
                    }
                ]
            })

    try:
        # create_trigger(request_body)
        print(request_body)
    except Exception as e:
        error += 1

    print(f"Total Error: {error}")



In [56]:

{
    'name': 'GA4 - Event(App) - 화면 조회',
    'type': 'cvt_90556056_474',
    'parameter': [
        {
            'type': 'boolean',
            'key': 'eventParamsCheck',
            'value': 'true'
        },
        {
            'type': 'template',
            'key': 'eventParamsVar',
            'value': '{{(App)GA4 - Params - 공유}}'
        },
        {
            'type': 'boolean', 
            'key': 'activePageInfo', 
            'value': 'false'
        }
    ],
    'tagFiringOption': 'oncePerEvent',
}

{'name': 'GA4 - Event(App) - 화면 조회',
 'type': 'cvt_90556056_474',
 'parameter': [{'type': 'boolean', 'key': 'eventParamsCheck', 'value': 'true'},
  {'type': 'template',
   'key': 'eventParamsVar',
   'value': '{{(App)GA4 - Params - 공유}}'},
  {'type': 'boolean', 'key': 'activePageInfo', 'value': 'false'}],
 'tagFiringOption': 'oncePerEvent'}

In [57]:
get_tag_list()

Listed triggers: [{'path': 'accounts/6003640738/containers/90556056/workspaces/43/tags/1094', 'accountId': '6003640738', 'containerId': '90556056', 'workspaceId': '43', 'tagId': '1094', 'name': 'CHTML - 검색어 쿠키 저장', 'type': 'html', 'parameter': [{'type': 'template', 'key': 'html', 'value': '<script>\n  var gas_now = new Date();\n  gas_now.setTime(gas_now.getTime() + (7 * 24 * 60 * 60 * 1000));\n  var gas_expires = ""expires="" + gas_now.toUTCString();\n  var gas_term = ""{{URL - 검색어 - keyword}}"";\n  var gas_shtm = ""shtm="" + encodeURI(gas_term) + ""; "";\n  document.cookie = gas_shtm + ""domain=.kyobobook.co.kr; "" + gas_expires + ""; path=/"";\n</script>'}, {'type': 'boolean', 'key': 'supportDocumentWrite', 'value': 'false'}], 'fingerprint': '1704680431091', 'firingTriggerId': ['831'], 'tagFiringOption': 'oncePerEvent', 'tagManagerUrl': 'https://tagmanager.google.com/#/container/accounts/6003640738/containers/90556056/workspaces/43/tags/1094?apiLink=tag', 'monitoringMetadata': {'type

{'tag': [{'path': 'accounts/6003640738/containers/90556056/workspaces/43/tags/1094',
   'accountId': '6003640738',
   'containerId': '90556056',
   'workspaceId': '43',
   'tagId': '1094',
   'name': 'CHTML - 검색어 쿠키 저장',
   'type': 'html',
   'parameter': [{'type': 'template',
     'key': 'html',
     'value': '<script>\n  var gas_now = new Date();\n  gas_now.setTime(gas_now.getTime() + (7 * 24 * 60 * 60 * 1000));\n  var gas_expires = ""expires="" + gas_now.toUTCString();\n  var gas_term = ""{{URL - 검색어 - keyword}}"";\n  var gas_shtm = ""shtm="" + encodeURI(gas_term) + ""; "";\n  document.cookie = gas_shtm + ""domain=.kyobobook.co.kr; "" + gas_expires + ""; path=/"";\n</script>'},
    {'type': 'boolean', 'key': 'supportDocumentWrite', 'value': 'false'}],
   'fingerprint': '1704680431091',
   'firingTriggerId': ['831'],
   'tagFiringOption': 'oncePerEvent',
   'tagManagerUrl': 'https://tagmanager.google.com/#/container/accounts/6003640738/containers/90556056/workspaces/43/tags/1094?apiL

In [70]:
sheet_names = get_sheet_name(xlsx)

for index, i in enumerate(sheet_names):
    print(index, i)

0 변수(당사쿠키)
1 변수(데이터 영역)
2 변수(URL)
3 변수(자동이벤트 변수)
4 변수(DOM 요소)
5 변수(맞춤JS)
6 변수(참고표)
7 변수(이벤트 매개변수)
8 트리거(맞춤 이벤트)
9 트리거(페이지뷰)
10 트리거(스크롤 깊이)
11 트리거(링크 클릭)(Web)
12 트리거(링크 클릭)(App)
13 트리거(모든 요소 클릭)(Web)
14 트리거(모든 요소 클릭)(App)
15 태그(맞춤HTML)
16  태그(Web)
17  태그(App 이벤트)


In [59]:
{
  'name': 'GA4 - Event - 공통 - 회원가입', 
  'type': 'gaawe', 
  'parameter': [
    {
      'type': 'template', 
      'key': 'measurementIdOverride', 
      'value': '{{GA4 - 측정 ID}}'
    }, 
    {
      'type': 'template', 
      'key': 'eventName', 
      'value': 'sign_up'
    }, 
    {
      'type': 'list', 
      'key': 'eventSettingsTable', 
      'list': [
        {
          'type': 'map', 
          'map': [
            {
              'type': 'template', 
              'key': 'parameter', 
              'value': 'method'
            }, 
            {
              'type': 'template', 
              'key': 'parameterValue', 
              'value': '{{DLV - method}}'
            }
          ]
        }, 
        {
          'type': 'map', 
          'map': [
            {
              'type': 'template', 
              'key': 'parameter', 
              'value': 'type'
            }, 
            {
              'type': 'template', 
              'key': 'parameterValue', 
              'value': '{{DLV - type}}'
            }
          ]
        }, 
        {
          'type': 'map', 
          'map': [
            {
              'type': 'template', 
              'key': 'parameter', 
              'value': 'user_id'
            }, 
            {
              'type': 'template', 
              'key': 'parameterValue', 
              'value': '{{DLV - user_id}}'
            }
          ]
        }, 
        {
          'type': 'map', 'map': [{'type': 'template', 'key': 'parameter', 'value': 'cm_count_signup'}, {'type': 'template', 'key': 'parameterValue', 'value': '1'}]}]}, 
      {
        'type': 'list', 
        'key': 'userProperties', 
        'list': [
          {
            'type': 'map', 
            'map': [
              {
                'type': 'template', 
                'key': 'name', 
                'value': 'up_signup_method'
              }, 
              {
                'type': 'template', 
                'key': 'value', 
                'value': '{{DLV - method}}'
              }
            ]
          }, 
          {
            'type': 'map', 'map': [{'type': 'template', 'key': 'name', 'value': 'up_user_type'}, {'type': 'template', 'key': 'value', 'value': '{{DLV - type}}'}]}]}], 'firingTriggerId': ['808']}


{'name': 'GA4 - Event - 공통 - 회원가입',
 'type': 'gaawe',
 'parameter': [{'type': 'template',
   'key': 'measurementIdOverride',
   'value': '{{GA4 - 측정 ID}}'},
  {'type': 'template', 'key': 'eventName', 'value': 'sign_up'},
  {'type': 'list',
   'key': 'eventSettingsTable',
   'list': [{'type': 'map',
     'map': [{'type': 'template', 'key': 'parameter', 'value': 'method'},
      {'type': 'template',
       'key': 'parameterValue',
       'value': '{{DLV - method}}'}]},
    {'type': 'map',
     'map': [{'type': 'template', 'key': 'parameter', 'value': 'type'},
      {'type': 'template',
       'key': 'parameterValue',
       'value': '{{DLV - type}}'}]},
    {'type': 'map',
     'map': [{'type': 'template', 'key': 'parameter', 'value': 'user_id'},
      {'type': 'template',
       'key': 'parameterValue',
       'value': '{{DLV - user_id}}'}]},
    {'type': 'map',
     'map': [{'type': 'template',
       'key': 'parameter',
       'value': 'cm_count_signup'},
      {'type': 'template', 'k

In [71]:
sheet_tag_web = read_sheet(xlsx, sheet_names[16])

create_tag_web(sheet_tag_web)

{'name': 'GA4 - Config', 'type': 'googtag', 'parameter': [{'type': 'template', 'key': 'measurementIdOverride', 'value': '{{GA4 - 측정 ID}}'}, {'type': 'template', 'key': 'eventSettingsVariable', 'value': '{{GA4 - Params - 공유}}'}, {'type': 'list', 'key': 'eventSettingsTable', 'list': []}, {'type': 'list', 'key': 'userProperties', 'list': []}], 'firingTriggerId': ['1096']}
{'name': 'GA4 - Event - 공통 - 회원가입', 'type': 'gaawe', 'parameter': [{'type': 'template', 'key': 'measurementIdOverride', 'value': '{{GA4 - 측정 ID}}'}, {'type': 'template', 'key': 'eventName', 'value': 'sign_up'}, {'type': 'list', 'key': 'eventSettingsTable', 'list': [{'type': 'map', 'map': [{'type': 'template', 'key': 'parameter', 'value': 'method'}, {'type': 'template', 'key': 'parameterValue', 'value': '{{DLV - method}}'}]}, {'type': 'map', 'map': [{'type': 'template', 'key': 'parameter', 'value': 'type'}, {'type': 'template', 'key': 'parameterValue', 'value': '{{DLV - type}}'}]}, {'type': 'map', 'map': [{'type': 'templa

/var/folders/xq/qf2k1k454qb_jfv62h4qfhn00000gn/T/ipykernel_72043/1283990398.py:80: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  'value': sheet.iloc[index][7]
/var/folders/xq/qf2k1k454qb_jfv62h4qfhn00000gn/T/ipykernel_72043/1283990398.py:97: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  'value': sheet.iloc[index][9]


KeyError: 'link - 종이책 - 상세 - 반품교환 클릭'